<a href="https://colab.research.google.com/github/Vimp17/py/blob/main/TransferLearningmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
path  = "/content/dogs"
train = True
p = path = os.path.join(path, "train/" if train else "test")

ResNet50 transferlearning on dogs pictures

In [ ]:
import os
import json
from PIL import Image

import torch
import torch.utils.data as data
from torchvision import models
import torchvision.transforms.v2 as tfs_v2
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm


class DogDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path, "train" if train else "test")
        self.transform = transform

        with open(os.path.join(self.path, "format.json"), "r") as fp:
            self.format = json.load(fp)

        self.length = 0
        self.files = []
        self.targets = torch.eye(10)

        for _dir, _target in self.format.items():
            path = os.path.join(self.path, _dir)
            list_files = os.listdir(path)
            self.length += len(list_files)
            self.files.extend(map(lambda _x: (os.path.join(path, _x), _target), list_files))

    def __getitem__(self, item):
        path_file, target = self.files[item]
        t = self.targets[target]
        img = Image.open(path_file)

        if self.transform:
            img = self.transform(img)

        return img, t

    def __len__(self):
        return self.length

resnet_weights = models.ResNet50_Weights.DEFAULT
transforms = resnet_weights.transforms()

model = models.resnet50(weights=resnet_weights)
model.requires_grad_(False)
model.fc = nn.Linear(512*4, 10)
model.fc.requires_grad_(True)

d_train = DogDataset("/content/dogs", transform=transforms)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)

optimizer = optim.Adam(params=model.fc.parameters(), lr=0.001, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()
epochs = 15
model.train()

for _e in range(epochs):
    loss_mean = 0
    lm_count = 0

    train_tqdm = tqdm(train_data,leave=True)
    for x_train, y_train in train_tqdm:
        predict = model(x_train)
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1/lm_count *loss.item() + (1 - 1/lm_count) * loss.item()
        train_tqdm.set_description(f"Epoch [{_e+1}/ {epochs}], loss_mean :{loss_mean:.4f}")

st = model.state_dict()
torch.save(st, "model_transfer_learning_resnet.tar")

d_test = DogDataset("/content/dogs", train=False, transform=transforms)
test_data = data.DataLoader(d_test, batch_size = 64, shuffle=False)

Q = 0
P = 0
count = 0
model.eval()

test_tqdm = tqdm(test_data, leave=True)
for x_test, y_test in test_tqdm:
    pre = model(x_test)
    p2 = torch.argmax(pre, dim=1)
    y = torch.argmax(y_test, dim=1)
    P += torch.sum(p2==y).item()
    Q += loss_func(pre, y_test).item()
    count+=1

Q /=count
P /= len(d_test)
print()
print(Q)
print(P)

Epoch [1/ 15], loss_mean :1.7394: 100%|██████████| 51/51 [07:30<00:00,  8.84s/it]
Epoch [2/ 15], loss_mean :0.6623: 100%|██████████| 51/51 [07:26<00:00,  8.75s/it]
Epoch [3/ 15], loss_mean :0.6007: 100%|██████████| 51/51 [07:26<00:00,  8.76s/it]
Epoch [4/ 15], loss_mean :1.0246: 100%|██████████| 51/51 [07:22<00:00,  8.68s/it]
Epoch [5/ 15], loss_mean :1.1563: 100%|██████████| 51/51 [07:30<00:00,  8.83s/it]
Epoch [6/ 15], loss_mean :0.8343: 100%|██████████| 51/51 [07:30<00:00,  8.84s/it]
Epoch [7/ 15], loss_mean :1.2935: 100%|██████████| 51/51 [07:34<00:00,  8.91s/it]
Epoch [8/ 15], loss_mean :0.7505: 100%|██████████| 51/51 [07:30<00:00,  8.84s/it]
Epoch [9/ 15], loss_mean :0.6558: 100%|██████████| 51/51 [07:31<00:00,  8.85s/it]
Epoch [10/ 15], loss_mean :1.3598: 100%|██████████| 51/51 [07:22<00:00,  8.68s/it]
Epoch [11/ 15], loss_mean :0.8213: 100%|██████████| 51/51 [07:17<00:00,  8.58s/it]
Epoch [12/ 15], loss_mean :0.5555: 100%|██████████| 51/51 [07:16<00:00,  8.57s/it]
Epoch [13/ 15], loss_mean :0.5248: 100%|██████████| 51/51 [07:33<00:00,  8.88s/it]
Epoch [14/ 15], loss_mean :0.6469: 100%|██████████| 51/51 [07:29<00:00,  8.82s/it]
Epoch [15/ 15], loss_mean :0.2556: 100%|██████████| 51/51 [07:29<00:00,  8.82s/it]
100%|██████████| 5/5 [01:27<00:00, 17.59s/it]
0.1835356295108795
0.9526813880126183